In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import pi
import plotly.express as px

In [2]:
def load_data(file_path, categories):
    data = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        model_name = None
        i=0
        for line in lines:
            if line.startswith("Model") or line.startswith(" Model"):
                model_name = line.split('[')[1].split(']')[0].strip()
                scores_part = line.split('=')[1].strip()
                for category in categories:
                    idx_start = scores_part.find(category) + len(category) + 1
                    idx_end = scores_part.find('%', idx_start)
                    score = int(scores_part[idx_start:idx_end].strip())
                    data.append({"model": model_name, "category": category, "score": score})
                    scores_part = scores_part[idx_end+1:]
    return data

file_path = 'GPT4.txt'
categories = [
        "Information Accuracy", "Guided Support", "Empathetic Validation",
        "Active Engagement", "Interpretation", "Probing for Clarity"
    ]
scores = load_data(file_path, categories)
df = pd.DataFrame(scores)

In [3]:
all_models = df["model"].unique()
scores_all = []
for model in all_models:
    for cat in categories:
        res = df[(df["category"]==cat) & (df["model"]==model)]
        score = res["score"].mean()
        scores_all.append({"model": model, "category": cat, "score": score})

In [11]:
# Filter and sort by target models
target_models = ["Vicuna-7b", "Alpaca-7b", "Llama2-7b", "Mamba-gpt", "Mistral-7b"]
scores_target = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models]
scores_target = sorted(scores_target, key=lambda x: target_models.index(x["model"]), reverse=True)

df_score = pd.DataFrame(scores_target)
df_score = df_score[df_score["model"].isin(target_models)]

rename_map = {
    "Vicuna-7b": "Vicuna-7B",
    "Alpaca-7b": "Alpaca-7B",
    "Llama2-7b": "LlaMa2-7B",
    "Mamba-gpt": "Mamba-GPT",
    "Mistral-7b": "Mistral-7B"
}

for k, v in rename_map.items():
    df_score.replace(k, v, inplace=True)

fig = px.line_polar(df_score, r = 'score', theta = 'category', line_close = True, category_orders = {"category": categories},
                    color = 'model', markers=True, color_discrete_sequence=px.colors.qualitative.Pastel, width=750, height=500)

fig.show()